<a href="https://colab.research.google.com/github/Nan2711/NEMO/blob/main/Huggingface_NMT_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets transformers[sentencepiece] sacrebleu

     |████████████████████████████████| 311 kB 29.9 MB/s 
     |████████████████████████████████| 3.8 MB 61.3 MB/s 
     |████████████████████████████████| 90 kB 9.5 MB/s 
     |████████████████████████████████| 1.1 MB 59.0 MB/s 
     |████████████████████████████████| 67 kB 7.1 MB/s 
     |████████████████████████████████| 134 kB 75.4 MB/s 
     |████████████████████████████████| 212 kB 76.7 MB/s 
     |████████████████████████████████| 271 kB 58.7 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 144 kB 73.8 MB/s 
     |████████████████████████████████| 596 kB 66.0 MB/s 
     |████████████████████████████████| 895 kB 66.7 MB/s 
     |████████████████████████████████| 6.5 MB 61.8 MB/s 
     |████████████████████████████████| 1.2 MB 60.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import transformers #hugging face transformer library 

print(transformers.__version__)

4.17.0


# Fine-tuning a model on a translation task

In [3]:
from datasets import list_datasets, load_dataset, list_metrics, load_metric

# Print all the available datasets
print(list_datasets())

['assin', 'ar_res_reviews', 'ambig_qa', 'bianet', 'ag_news', 'ajgt_twitter_ar', 'aeslc', 'bc2gm_corpus', 'air_dialogue', 'acronym_identification', 'afrikaans_ner_corpus', 'allegro_reviews', 'ade_corpus_v2', 'adversarial_qa', 'alt', 'billsum', 'amazon_polarity', 'amttl', 'ascent_kb', 'big_patent', 'bn_hate_speech', 'bswac', 'capes', 'arsentd_lev', 'bible_para', 'amazon_reviews_multi', 'ai2_arc', 'beans', 'anli', 'arabic_pos_dialect', 'best2009', 'boolq', 'ar_sarcasm', 'asnq', 'bnl_newspapers', 'amazon_us_reviews', 'arxiv_dataset', 'banking77', 'bookcorpus', 'bprec', 'c3', 'allocine', 'aslg_pc12', 'biosses', 'bbc_hindi_nli', 'americas_nli', 'biomrc', 'blimp', 'aqua_rat', 'aquamuse', 'ar_cov19', 'arabic_speech_corpus', 'arcd', 'blended_skill_talk', 'cail2018', 'blbooksgenre', 'bookcorpusopen', 'bsd_ja_en', 'atomic', 'autshumato', 'blog_authorship_corpus', 'caner', 'app_reviews', 'arabic_billion_words', 'art', 'asset', 'blbooks', 'brwac', 'c4', 'ami', 'assin2', 'babi_qa', 'bbaw_egyptian', 

In [4]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

## Loading the dataset

In [5]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("ai4bharat/samanantar",'hi') #largest publicly available parallel corpora collection for Indic languages
metric = load_metric("sacrebleu") # loading an existing metric

Downloading:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset samanantar downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___samanantar/hi/0.3.0/556308f80c011cb3c32f3de18199d7b1e4cf9ca707843c92bb0bede0e47a8bd6. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

In [6]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['idx', 'src', 'tgt', 'data_source'],
        num_rows: 8314505
    })
})

In [7]:
raw_datasets["train"][0]

{'data_source': 'ie_sports',
 'idx': 0,
 'src': 'Rayudus crucial knock comprised eight fours and four sixes.',
 'tgt': 'रायुडू की पारी में आठ चौके और चार छक्के शामिल हैं।'}

In [8]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(raw_datasets["train"])

,idx,src,tgt,data_source
0,4518,the Custom Officers were,कस्टम अधिकारी थे,anuvaad_nouns_dictionary
1,1681003,What is amazing is ...,चौकाने वाली बात यह है की .,indiccorp
2,87380,"Therefore, during President Trumps visit to India, established norms and protocols for handling such high level visits were followed by the Government of India, he wrote","उन्होंने आगे कहा, ‘इसलिए राष्ट्रपति ट्रंप की भारत यात्रा के दौरान भारत सरकार द्वारा इस तरह की उच्चस्तरीय यात्राओं के संचालन के लिए स्थापित मानकों और प्रोटोकॉल का अनुपालन किया गया था",thewire
3,3387020,Former India captain Mohammad Azharuddin has filed his nomination for the presidents post of the Hyderabad Cricket Association.,पूर्व भारतीय कप्तान मोहम्मद अज़हरुद्दीन हैदराबाद क्रिकेट एसोसिएशन के अध्यक्ष हैं.,indiccorp
4,443556,Professional skill training in schools,स्कूलों में व्यावसायिक शिक्षा की तैयारी,indiccorp


The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [10]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: The system stream (a sequence of segments).
    references: A list of one or more reference streams (each a sequence of segments).
    smooth_method: The smoothing method to use. (Default: 'exp').
    smooth_value: The smoothing value. Only valid for 'floor' and 'add-k'. (Defaults: floor: 0.1, add-k: 1).
    tokenize: Tokenization method to use for BLEU. If not provided, defaults to 'zh' for Chinese, 'ja-mecab' for
        Japanese and '13a' (mteval) otherwise.
    lowercase: Lowercase the data. If True, enables case-insensitivity. (Default: False).
    force: Insist that your tokenized input is actually detokenized.

Returns:
    'score': BLEU score,
    'counts'

In [11]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'bp': 1.0,
 'counts': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'ref_len': 4,
 'score': 0.0,
 'sys_len': 4,
 'totals': [4, 2, 0, 0]}

## Preprocessing the data

In [12]:
from transformers import AutoTokenizer
#load models without needing to set which transformer architecture or tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/793k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00M [00:00<?, ?B/s]

In [13]:
if "mbart" in model_checkpoint:
    tokenizer.src_lang = "en-XX"
    tokenizer.tgt_lang = "hi_IN"

In [14]:
tokenizer("Hello, this one sentence!")

{'input_ids': [12110, 2, 90, 131, 8800, 61, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [15]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[12110, 2, 90, 131, 8800, 61, 0], [239, 23, 414, 8800, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [16]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[2204, 10967, 818, 2, 90, 44, 4625, 44, 16, 4072, 1936, 5386, 61, 0], [44, 1687, 23, 44, 980, 9972, 581, 44, 16, 4072, 1936, 5386, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [17]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Hindi: "
else:
    prefix = ""

In [18]:
'''max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "hi"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["src"]]
    targets = [ex[target_lang] for ex in examples["tgt"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs'''

'max_input_length = 128\nmax_target_length = 128\nsource_lang = "en"\ntarget_lang = "hi"\n\ndef preprocess_function(examples):\n    inputs = [prefix + ex[source_lang] for ex in examples["src"]]\n    targets = [ex[target_lang] for ex in examples["tgt"]]\n    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)\n\n    # Setup the tokenizer for targets\n    with tokenizer.as_target_tokenizer():\n        labels = tokenizer(targets, max_length=max_target_length, truncation=True)\n\n    model_inputs["labels"] = labels["input_ids"]\n    return model_inputs'

In [19]:
max_input_length = 32
max_target_length = 32
source_lang = "src"
target_lang = "tgt"

def preprocess_function(examples):
    inputs = examples["src"]
    targets = examples["tgt"]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [20]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[34296, 9780, 2454, 16835, 24098, 50318, 291, 2941, 1278, 16, 10, 1278, 44032, 3, 0], [9205, 31628, 12756, 21, 10, 87, 3401, 27810, 4, 4556, 5331, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[5054, 3625, 31543, 15, 37401, 11, 4495, 14111, 174, 9, 1634, 6712, 37577, 1130, 28, 40, 0], [289, 37, 15629, 139, 9, 205, 578, 13293, 24, 2711, 7171, 57, 324, 82, 40, 0]]}

In [21]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/8315 [00:00<?, ?ba/s]

In [22]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['idx', 'src', 'tgt', 'data_source', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 8314505
    })
})

## Fine-tuning the model

In [23]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint) #This is a generic model class that will be instantiated as one of the model classes of the library (with a sequence-to-sequence language modeling head) 

Downloading:   0%|          | 0.00/292M [00:00<?, ?B/s]

In [24]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    #fp16=True,
    #push_to_hub=True,
)

In [25]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [26]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [27]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    #eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: data_source, src, idx, tgt. If data_source, src, idx, tgt are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 8314505
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 519657


Epoch,Training Loss,Validation Loss
